In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("nba_games.csv", index_col=0)

In [8]:
df

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,38.0,72.0,0.528,16.0,37.0,0.432,18.0,21.0,...,26.2,26.8,155.0,123.0,MIA,107,1,2023,2022-11-07,True
1,240.0,240.0,40.0,84.0,0.476,14.0,39.0,0.359,13.0,15.0,...,41.0,37.3,160.0,121.0,POR,110,0,2023,2022-11-07,False
2,240.0,240.0,41.0,78.0,0.526,8.0,24.0,0.333,15.0,19.0,...,28.6,41.1,250.0,125.0,DAL,90,1,2023,2022-12-14,True
3,240.0,240.0,29.0,74.0,0.392,13.0,38.0,0.342,19.0,26.0,...,12.6,33.0,183.0,110.0,CLE,105,0,2023,2022-12-14,False
4,240.0,240.0,37.0,87.0,0.425,7.0,33.0,0.212,32.0,35.0,...,20.0,32.3,226.0,117.0,TOR,126,1,2023,2022-12-07,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20437,240.0,240.0,33.0,85.0,0.388,12.0,44.0,0.273,28.0,34.0,...,51.5,36.2,141.0,114.0,BOS,117,0,2020,2020-09-19,False
20438,240.0,240.0,43.0,91.0,0.473,17.0,45.0,0.378,23.0,32.0,...,53.2,31.1,225.0,130.0,UTA,124,1,2025,2024-10-23,True
20439,240.0,240.0,40.0,85.0,0.471,10.0,26.0,0.385,34.0,45.0,...,22.5,34.2,258.0,125.0,MEM,126,0,2025,2024-10-23,False
20440,240.0,240.0,42.0,82.0,0.512,12.0,27.0,0.444,12.0,19.0,...,53.2,28.0,146.0,126.0,TOR,121,1,2024,2023-11-24,False


In [9]:
df = df.sort_values("date")

In [10]:
df = df.reset_index(drop=True)

In [11]:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

In [12]:
def add_target(group):
    group["target"] = group["won"].shift(-1)
    return group

df = df.groupby("team", group_keys=False).apply(add_target)

/var/folders/y6/097q2jgx0y38117lgk3j9nf40000gn/T/ipykernel_72482/3080091488.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  group["target"] = group["won"].shift(-1)
/var/folders/y6/097q2jgx0y38117lgk3j9nf40000gn/T/ipykernel_72482/3080091488.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  group["target"] = group["won"].shift(-1)
/var/folders/y6/097q2jgx0y38117lgk3j9nf40000gn/T/ipykernel_72482/3080091488.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many tim

In [13]:
df[df["team"] == "WAS"]

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
17,240.0,42.0,97.0,0.433,6.0,22.0,0.273,30.0,38.0,0.789,...,31.2,148.0,125.0,PHI,115,0,2018,2017-10-18,True,True
43,240.0,40.0,75.0,0.533,6.0,17.0,0.353,29.0,35.0,0.829,...,43.6,215.0,122.0,DET,111,0,2018,2017-10-20,True,True
83,240.0,37.0,84.0,0.440,9.0,30.0,0.300,26.0,28.0,0.929,...,35.1,272.0,115.0,DEN,104,1,2018,2017-10-23,True,False
127,265.0,39.0,95.0,0.411,6.0,26.0,0.231,15.0,23.0,0.652,...,32.9,129.0,98.0,LAL,102,1,2018,2017-10-25,False,False
139,240.0,43.0,97.0,0.443,15.0,33.0,0.455,16.0,22.0,0.727,...,30.9,210.0,122.0,GSW,120,1,2018,2017-10-27,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20139,240.0,35.0,86.0,0.407,12.0,45.0,0.267,8.0,10.0,0.800,...,26.9,208.0,112.0,BOS,124,1,2025,2025-04-06,False,False
20161,240.0,38.0,91.0,0.418,11.0,39.0,0.282,11.0,15.0,0.733,...,26.6,145.0,112.0,IND,104,1,2025,2025-04-08,False,False
20180,240.0,34.0,87.0,0.391,10.0,33.0,0.303,25.0,30.0,0.833,...,31.5,165.0,109.0,PHI,122,0,2025,2025-04-09,False,False
20227,240.0,35.0,91.0,0.385,10.0,41.0,0.244,9.0,12.0,0.750,...,31.2,261.0,97.0,CHI,119,1,2025,2025-04-11,False,True


In [14]:
df["target"][pd.isnull(df["target"])] = 2
df["target"] = df["target"].astype(int, errors="ignore")

/var/folders/y6/097q2jgx0y38117lgk3j9nf40000gn/T/ipykernel_72482/2793819745.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["target"][pd.isnull(df["target"])] = 2
/var/folders/y6/097q2jgx0y38117lgk3j9nf40000gn/T/ipykernel_72482/279381

In [15]:
df["won"].value_counts()

won
True     10221
False    10221
Name: count, dtype: int64

In [16]:
df["target"].value_counts()

target
1    10206
0    10206
2       30
Name: count, dtype: int64

In [17]:
nulls = pd.isnull(df)

In [18]:
nulls = nulls.sum()

In [19]:
nulls= nulls[nulls > 0]

In [20]:
nulls

ft%                 1
gmsc            20442
+/-             20442
mp_max          20442
mp_max.1        20442
ft%_max             1
+/-_max            15
ft%_opp             1
gmsc_opp        20442
+/-_opp         20442
mp_max_opp      20442
mp_max_opp.1    20442
ft%_max_opp         1
+/-_max_opp        15
dtype: int64

In [21]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [22]:
df=  df[valid_columns].copy()

In [23]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier
rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction="forward", cv=split )

In [24]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]

In [26]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [28]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [29]:
sfs.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [31]:
predictors = list(selected_columns[sfs.get_support()])

In [38]:
def backtest(data, model, predictors, start=2, step=1):
  all_predictions = []

  seasons = sorted(data["season"].unique())

  for i in range(start, len(seasons), step):
    season = seasons[i]

    train = data[data["season"] < season] 
    test = data[data["season"] == season]

    model.fit(train[predictors], train["target"])

    preds = model.predict(test[predictors])
    preds = pd.Series(preds,index=test.index)

    combined = pd.concat([test["target"], preds], axis=1)
    combined.columns = ["actual", "prediction"]

    all_predictions.append(combined)

  return pd.concat(all_predictions)

In [39]:
predictions = backtest(df, rr, predictors)

In [41]:
predictions

,actual,prediction
5248,0,1
5249,1,1
5250,1,1
5251,0,0
5252,0,1
...,...,...
20437,0,1
20438,1,0
20439,0,1
20440,2,1
